In [76]:
import ee
import json
import time
import pandas as pd

# # Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [77]:
def import_aois(csv_loc, Full_Congo_Pull=False):
    
    features = []
    
    if Full_Congo_Pull:
        with open(csv_loc,"r",encoding='utf-8') as jsonfile:
            data = json.load(jsonfile)
            for geometry in data["features"]:
                polygon = geometry["geometry"]["coordinates"][0][0]
                poly_obj = ee.Geometry.Polygon(polygon)
                feature = ee.Feature(poly_obj)
                features.append(feature)
                
            
            return ee.FeatureCollection(features)
    else:
        df_labels = pd.read_csv(csv_loc)
        df_polygons = df_labels["polygon"]

        for geometry in df_labels["polygon"]:
            polygon = json.loads(geometry)["coordinates"]
            poly_obj = ee.Geometry.Polygon(polygon)
            feature = ee.Feature(poly_obj)
            features.append(feature)
            
        return ee.FeatureCollection(features)
        
zhenya_misha_polygon_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/rasters/Tiles_v3/Polygon_List/polygons_101320.csv"        
zhenya_grid_path_2 = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/congo_basin_boundary/Congo_Basin_Boundary_no_islands_v5_gridded.geojson"

In [78]:
fc = import_aois(zhenya_grid_path_2, Full_Congo_Pull=True)

In [79]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [80]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [81]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [82]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

In [83]:
def apply_cld_shdw_mask_all_bands(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.updateMask(not_cld_shdw)

In [84]:
def add_ndvi(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    img = img.addBands(ndvi)
    img = img.float()
    return img

In [85]:
# def add_ndvi_rescaled(img):
#     ndvi = img.normalizedDifference(['B8', 'B4'])
#     ndvi_rescaled = ndvi.multiply(10).rename('NDVI_rescaled')
#     img = img.addBands(ndvi_rescaled)
#     return img

In [86]:
START_DATE = '2019-01-01'
END_DATE = '2020-12-31'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100
band_list = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12','TCI_R','TCI_G','TCI_B','AOT','WVP']

In [87]:
def export_to_gcs(s2_sr_median,AOI,polygon_id,band_list):
    
    time_stamp = "_".join(time.ctime().split(" ")[1:])
    time_stamp = time_stamp.replace(':', '_')
    
    export = ee.batch.Export.image.toCloudStorage(
      image=s2_sr_median.select(band_list),
      description=f'{str(polygon_id)}_full_band_s2cloudless_export',
      scale=10,
      region=AOI,
      fileNamePrefix=f'S2_CloudFree/full_congo_s2cloudless_3/{str(polygon_id)}_{time_stamp}',
      bucket='project-canopy-temp-2',
      maxPixels=1e13
)
    export.start()
    
    return export

In [88]:
def s2cloudless_process_download(feature_list, sort_by='id', band_list=band_list, add_NDVI = False):
    
    feature_list = feature_list.sort(sort_by).toList(feature_list.size())
    n_features = feature_list.size().getInfo()

    print(f'{n_features} features have been loaded')

    for i in range(0, n_features):
        polygon_id = i+1
        
        print("processing ",polygon_id)

        feature_point = ee.Feature( feature_list.get(i) )

        AOI = feature_point.geometry()
        
        s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)
        
        s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                         .map(apply_cld_shdw_mask_all_bands)
                         .median())
        
        if add_NDVI:
        
            s2_sr_median = add_ndvi(s2_sr_median)
            
        else:
            
            s2_sr_median = s2_sr_median.toInt16()
            
        
        s2_sr_median = s2_sr_median.clip(AOI).reproject('EPSG:4326', None, 10)

        export_to_gcs(s2_sr_median,AOI,polygon_id=polygon_id,band_list=band_list)
        
        

In [89]:
s2cloudless_process_download(feature_list = fc, sort_by='id', band_list=band_list)

2403 features have been loaded
processing  1
processing  2
processing  3
processing  4
processing  5
processing  6
processing  7
processing  8
processing  9
processing  10
processing  11
processing  12
processing  13
processing  14
processing  15
processing  16
processing  17
processing  18
processing  19
processing  20
processing  21
processing  22
processing  23
processing  24
processing  25
processing  26
processing  27
processing  28
processing  29
processing  30
processing  31
processing  32
processing  33
processing  34
processing  35
processing  36
processing  37
processing  38
processing  39
processing  40
processing  41
processing  42
processing  43
processing  44
processing  45
processing  46
processing  47
processing  48
processing  49
processing  50
processing  51
processing  52
processing  53
processing  54
processing  55
processing  56
processing  57
processing  58
processing  59
processing  60
processing  61
processing  62
processing  63
processing  64
processing  65
pro

# Sandbox

In [28]:
def s2cloudless_process_download(feature_list, sort_by='id', band_list=band_list):
    
    feature_list = feature_list.sort(sort_by).toList(feature_list.size())
    n_features = feature_list.size().getInfo()

    print(f'{n_features} features have been loaded')

    for i in range(0, n_features):
        polygon_id = i+1
        
        print("processing ",polygon_id)

        feature_point = ee.Feature( feature_list.get(i) )

        AOI = feature_point.geometry()
        
        s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)
        
        s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                         .map(apply_cld_shdw_mask_all_bands)
                         .median())
        
        s2_sr_median = add_ndvi(s2_sr_median)
        
        s2_sr_median = s2_sr_median.clip(AOI).reproject('EPSG:4326', None, 10)

#         export_to_gcs(s2_sr_median,AOI,polygon_id=polygon_id,band_list=band_list)
        
        

In [29]:
col = s2cloudless_process_download(feature_list = fc, sort_by='id', band_list=band_list)

2403 features have been loaded
processing  1


In [31]:
col.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [11133, 11133],
   'origin': [77923, -66792],
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    0,
    0,
    -8.983152841195215e-05,
    0]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [11133, 11133],
   'origin': [77923, -66792],
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    0,
    0,
    -8.983152841195215e-05,
    0]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [11133, 11133],
   'origin': [77923, -66792],
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    0,
    0,
    -8.983152841195215e-05,
    0]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [11133, 11133],
   'origin': [77923, -66792],
   'crs': 'EPSG:4326',
   'crs_transform':